**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [1]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json

from keras.models import Sequential, model_from_json
from keras.layers.core import Dense
from keras.optimizers import sgd
from keras.layers import Flatten
from keras.layers import Conv2D, MaxPooling2D, Dropout, Activation, AveragePooling2D,Reshape,BatchNormalization

/home/ali/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# MiniProject #3: Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [2]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [3]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

the function act chose randomly between a random action with probability $\epsilon$, or the best action drawn from the current policy with probability $1 - \epsilon$ to return.


The parameter $\epsilon$ is essential to add exploration of other reward in our model.

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [4]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [18]:
# parameters
size = 13
T=200
temperature=0.5
epochs_train=30 # set small when debugging
epochs_test=10 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

Array position store the state $s_t$(position at time t).

Array board store the rewards estimate at time t drawn from the current policy

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [6]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        return np.random.randint(0, self.n_action)

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [7]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):
        # To assume that the games will end
        Play = True
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        
        win = 0
        lose = 0
        
        while Play:
            # The agent chosses an action
            next_action = agent.learned_act(state)
            # Apply this action to the environment
            # than get the next state and the reward
            # and if the games end
            last = state
            state, reward, game_over = env.act(next_action)
            Play = not game_over
            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward
            # Apply the reinforcement strategy
            loss = agent.reinforce(last, state,  next_action, reward, game_over)
        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [19]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random0.mp4'))

Win/lose count 17.5/22.0. Average score (-4.5)
Win/lose count 18.5/19.0. Average score (-2.5)
Win/lose count 17.5/21.0. Average score (-2.8333333333333335)
Win/lose count 16.0/11.0. Average score (-0.875)
Win/lose count 13.0/19.0. Average score (-1.9)
Win/lose count 17.5/19.0. Average score (-1.8333333333333333)
Win/lose count 19.0/20.0. Average score (-1.7142857142857142)
Win/lose count 19.0/18.0. Average score (-1.375)
Win/lose count 11.5/9.0. Average score (-0.9444444444444444)
Win/lose count 11.5/14.0. Average score (-1.1)
Final score: -1.1


***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}





Let $\pi(.|s)$ denote the policy distribution, and $p(.|s,a)$ the transition probability distribution of states. The distribution of the couple $(s',a')$ given $(s,a)$ is given by :
\begin{equation*}
p(s',a'|s,a) = \pi(a'|s)p(s'|s,a)
\end{equation*}
the $Q$ function is expressed as:

\begin{align*}
Q^\pi(s,a) &= E_{p^{\pi}}[\sum_{t = 0}^{\infty}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \\
&= r(s,a) + \gamma E_{p^{\pi}}[\sum_{t = 0}^{\infty}\gamma^{t}r(s_{t+1},a_{t+1})|s_{0}=s,a_{0}=a] \\
&= r(s,a) + \gamma \sum_{(s',a')} \mathbb{P}(s_1 = s'|s_0 = s, a_0=a) \pi(a_0 = a'|s_0=s) E_{p^{\pi}}[\sum_{t = 0}^{\infty}\gamma^{t}r(s_{t},a_{t})|s_{0}=s',a_{0}=a'] \\
&= r(s,a) + \gamma \sum_{(s',a')} p(s',a'|s,a) Q^{\pi}(s',a') \\
&= r(s,a) + \gamma E_{(s',a')\sim p(.|s,a)}[Q^{\pi}(s',a')] \\
&= E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{align*}



We now derive the expression of $Q^*$:

\begin{align*}
Q^*(s,a) &= \max_{\pi} Q^{\pi}(s,a) \\
&= \max_{\pi} E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')] \\
&= E_{s' \sim p(.|s, \pi^*(s|a))}[r(s,a) + \gamma \max_{a'} Q^*(s',a') ]
\end{align*}





***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [11]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        self.memory.append(m)
        if len(self.memory)==self.max_memory:
            self.memory.pop(0)
            

    def random_access(self):
        index = np.random.choice(range(len(self.memory)))
        return self.memory[index]

***
The pipeline we will use for training is given below:

In [12]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [13]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        return np.argmax(self.model.predict(s.reshape([1] + list(s.shape))))

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        shape_state = [1] + list(s_.shape)
        
        for i in range(self.batch_size):
            [state, next_state, action, reward, game_over] = self.memory.random_access()
            
            target_q[i] = self.model.predict(state.reshape(shape_state))
            
            if game_over:
                target_q[i, action] = reward
            else:
                target_q[i, action] = reward + self.discount * self.model.predict(
                    next_state.reshape(shape_state)).max()
            input_states[i] = state
        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)

        l = self.model.train_on_batch(input_states, target_q)


        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1, input_shape=(5, 5, 2),**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # NN Model
        model=Sequential()
        model.add(Flatten(input_shape=input_shape)) 
        model.add(Dense(32, activation='relu'))
        model.add(Dropout(0.15))
        model.add(Dense(16, activation='relu'))
        model.add(Dropout(0.3))
        model.add(Dense(4, activation='sigmoid'))
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model
        

In [21]:
env = Environment(grid_size=size, max_time=T, temperature=0.5)
agent = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train10.mp4'))

Epoch 000/030 | Loss 0.0212 | Win/lose count 6.0/3.0 (3.0)
Epoch 001/030 | Loss 0.0224 | Win/lose count 14.0/18.0 (-4.0)
Epoch 002/030 | Loss 0.0105 | Win/lose count 3.0/6.0 (-3.0)
Epoch 003/030 | Loss 0.0085 | Win/lose count 4.5/4.0 (0.5)
Epoch 004/030 | Loss 0.0255 | Win/lose count 6.5/6.0 (0.5)
Epoch 005/030 | Loss 0.0103 | Win/lose count 8.0/5.0 (3.0)
Epoch 006/030 | Loss 0.0139 | Win/lose count 8.0/9.0 (-1.0)
Epoch 007/030 | Loss 0.0339 | Win/lose count 7.0/5.0 (2.0)
Epoch 008/030 | Loss 0.0170 | Win/lose count 3.0/7.0 (-4.0)
Epoch 009/030 | Loss 0.0070 | Win/lose count 3.5/1.0 (2.5)
Epoch 010/030 | Loss 0.0136 | Win/lose count 9.0/7.0 (2.0)
Epoch 011/030 | Loss 0.0211 | Win/lose count 6.0/4.0 (2.0)
Epoch 012/030 | Loss 0.0182 | Win/lose count 4.5/5.0 (-0.5)
Epoch 013/030 | Loss 0.0202 | Win/lose count 3.0/2.0 (1.0)
Epoch 014/030 | Loss 0.0185 | Win/lose count 4.0/6.0 (-2.0)
Epoch 015/030 | Loss 0.0236 | Win/lose count 5.0/1.0 (4.0)
Epoch 016/030 | Loss 0.0050 | Win/lose count 8.0

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [24]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1, input_shape=(5,5,2), **kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        model=Sequential()
        model.add(Conv2D(8,(2,2),activation="relu",input_shape=input_shape,padding='same'))
        model.add(Dropout(0.15))
        model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))
        model.add(Conv2D(8,(2,2),activation="relu",input_shape=input_shape,padding='same'))
        model.add(Dropout(0.3))
        model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))
        model.add(Flatten())
        model.add(Dense(4,activation="sigmoid"))
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [25]:
env = Environment(grid_size=size, max_time=T, temperature=0.5)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent,env,epochs_train,prefix='cnn_train')
HTML(display_videos('cnn_train10.mp4'))

Epoch 000/030 | Loss 0.0023 | Win/lose count 5.0/1.0 (4.0)
Epoch 001/030 | Loss 0.0023 | Win/lose count 6.0/7.0 (-1.0)
Epoch 002/030 | Loss 0.0137 | Win/lose count 3.0/5.0 (-2.0)
Epoch 003/030 | Loss 0.0176 | Win/lose count 6.5/6.0 (0.5)
Epoch 004/030 | Loss 0.0120 | Win/lose count 4.5/3.0 (1.5)
Epoch 005/030 | Loss 0.0102 | Win/lose count 1.0/2.0 (-1.0)
Epoch 006/030 | Loss 0.0099 | Win/lose count 7.5/4.0 (3.5)
Epoch 007/030 | Loss 0.0002 | Win/lose count 8.5/3.0 (5.5)
Epoch 008/030 | Loss 0.0078 | Win/lose count 5.5/3.0 (2.5)
Epoch 009/030 | Loss 0.0047 | Win/lose count 5.5/5.0 (0.5)
Epoch 010/030 | Loss 0.0124 | Win/lose count 8.0/3.0 (5.0)
Epoch 011/030 | Loss 0.0040 | Win/lose count 6.5/4.0 (2.5)
Epoch 012/030 | Loss 0.0278 | Win/lose count 6.5/6.0 (0.5)
Epoch 013/030 | Loss 0.0020 | Win/lose count 2.5/1.0 (1.5)
Epoch 014/030 | Loss 0.0039 | Win/lose count 6.5/3.0 (3.5)
Epoch 015/030 | Loss 0.0020 | Win/lose count 5.0/4.0 (1.0)
Epoch 016/030 | Loss 0.0095 | Win/lose count 3.5/4.0 

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [26]:
env = Environment(grid_size=size, max_time=T,temperature=0.5)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Test of the CNN
Win/lose count 0/0. Average score (0.0)
Win/lose count 2.0/0. Average score (1.0)
Win/lose count 0.5/1.0. Average score (0.5)
Win/lose count 0.5/1.0. Average score (0.25)
Win/lose count 0.5/0. Average score (0.3)
Win/lose count 0.5/0. Average score (0.3333333333333333)
Win/lose count 1.0/0. Average score (0.42857142857142855)
Win/lose count 0.5/0. Average score (0.4375)
Win/lose count 2.5/0. Average score (0.6666666666666666)
Win/lose count 0/1.0. Average score (0.5)
Final score: 0.5
Test of the FC
Win/lose count 2.0/0. Average score (2.0)
Win/lose count 2.0/1.0. Average score (1.5)
Win/lose count 2.0/1.0. Average score (1.3333333333333333)
Win/lose count 0.5/0. Average score (1.125)
Win/lose count 1.0/1.0. Average score (0.9)
Win/lose count 1.5/2.0. Average score (0.6666666666666666)
Win/lose count 1.5/1.0. Average score (0.6428571428571429)
Win/lose count 2.5/1.0. Average score (0.75)
Win/lose count 1.0/0. Average score (0.7777777777777778)
Win/lose count 0.5/3.0. Ave

In [27]:
HTML(display_videos('cnn_test10.mp4'))

In [28]:
HTML(display_videos('fc_test10.mp4'))

I observe that The agent tends to not explore the map, which can be problematic for both methods. 
The temperature impacte the distribution of malus and bonus in the map, if the temperature increases the number of bonus increases and the number of malus decreases, than is more easy for the agent to perform better scroe.

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [29]:
def train_explore(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0
        while not game_over:
            # The agent performs an action
            action = agent.act(state)
            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)
            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < -0.1:
                lose = lose -reward
            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')
def test_explore(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):
        # To assume that the games will end
        Play = True
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        
        win = 0
        lose = 0
        
        while Play:
            # The agent chosses an action
            next_action = agent.learned_act(state)
            # Apply this action to the environment
            # than get the next state and the reward
            # and if the games end
            last = state
            state, reward, game_over = env.act(next_action)
            Play = not game_over
            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward
            # Apply the reinforcement strategy
            loss = agent.reinforce(last, state,  next_action, reward, game_over)
        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))
        
class EnvironmentExploring(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))
        
        self.malus_position = np.zeros((grid_size,grid_size))

    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        reward -= self.malus_position[self.x, self.y]
        self.board[self.x, self.y] = 0
        self.malus_position[self.x, self.y] = 0.1
        game_over = self.t > self.max_time
        # 3 "feature" states instead of 2
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1),
                        self.malus_position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]
        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""
    
        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1),
                        self.malus_position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state

In [31]:
# Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.5)
agent = DQN_CNN(size, lr=.1, epsilon = 0.2, memory_size=2000, batch_size = 32,n_state=3, input_shape=(5,5,3))
train_explore(agent, env, epochs_train, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore10.mp4'))

Epoch 000/030 | Loss 0.0062 | Win/lose count 9.5/5.0 (4.5)
Epoch 001/030 | Loss 0.0154 | Win/lose count 8.5/10.0 (-1.5)
Epoch 002/030 | Loss 0.0280 | Win/lose count 10.200000000000001/7.1 (3.1000000000000014)
Epoch 003/030 | Loss 0.0253 | Win/lose count 5.5/7.5 (-2.0)
Epoch 004/030 | Loss 0.0105 | Win/lose count 2.5/10.899999999999999 (-8.399999999999999)
Epoch 005/030 | Loss 0.0252 | Win/lose count 11.100000000000005/9.7 (1.4000000000000057)
Epoch 006/030 | Loss 0.0071 | Win/lose count 4.3999999999999995/11.999999999999998 (-7.599999999999999)
Epoch 007/030 | Loss 0.0304 | Win/lose count 1.2000000000000002/4.4 (-3.2)
Epoch 008/030 | Loss 0.0249 | Win/lose count 7.600000000000001/5.4 (2.200000000000001)
Epoch 009/030 | Loss 0.0156 | Win/lose count 8.400000000000002/7.699999999999999 (0.7000000000000028)
Epoch 010/030 | Loss 0.0118 | Win/lose count 4.2/8.799999999999999 (-4.599999999999999)
Epoch 011/030 | Loss 0.0073 | Win/lose count 8.800000000000002/5.5 (3.3000000000000025)
Epoch 012

In [32]:
# Evaluation
test_explore(agent,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore10.mp4'))

Win/lose count 2.8/24.400000000000084. Average score (-21.600000000000083)
Win/lose count 2.4/21.500000000000036. Average score (-20.35000000000006)
Win/lose count 0.8/21.90000000000004. Average score (-20.600000000000055)
Win/lose count 2.4/21.500000000000043. Average score (-20.22500000000005)
Win/lose count 3.1999999999999997/25.200000000000095. Average score (-20.58000000000006)
Win/lose count 2.4/26.500000000000096. Average score (-21.166666666666732)
Win/lose count 2.4/22.50000000000005. Average score (-21.014285714285773)
Win/lose count 1.6/19.70000000000001. Average score (-20.650000000000055)
Win/lose count 0/20.100000000000016. Average score (-20.588888888888942)
Win/lose count 2.0/21.60000000000004. Average score (-20.49000000000005)
Final score: -20.49000000000005


***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

***